### Load data

- ST: SESSION --> START, TIIME_DELTA
- SRM: ROOM, SESSION --> TYPE
- SRP: SESSION, ROOM, S_ORDER --> FIRST_NAME, LAST_NAME

In [45]:
from urllib.request import urlretrieve

URLS = {
    "ST.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=1165578014", 
    "SRM.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=244777042", 
    "SRP.csv": "https://docs.google.com/spreadsheets/d/1IQkcuu7cRXA5wlwMJspwSSfZNHIdXoapZBKufdFIayc/export?format=csv&gid=1456705243" 
}

for filename, url in URLS.items():
    urlretrieve(url, filename)

In [71]:
# global DataFrame
import pandas as pd

SESSION_NAMES = {
    "Reg": "Registration", 
    "OR": "Opening Remarks", 
    "Rec": "Reception", 
    "Photo": "Photo", 
    "BM": "Business Meeting", 
    "E": "Excursion", 
    "Ban": "Banquet", 
    "CR": "Closing Remarks"
}
for s,num,name in [("P",10,"Plenary Talk"), ("CB",8,"Coffee Break"), ("S",11,"Mini-symposium"), ("LB",4,"Lunch Break")]:
    for t in range(1, num + 1):
        SESSION_NAMES[s + "%d"%t] = name

st = pd.read_csv("ST.csv")
st["START"] = pd.to_datetime(st.START, format="%Y-%m-%d %H:%M")
st["TIME_DELTA"] = pd.to_timedelta(st.TIME_DELTA, unit="m")
st.insert(2, "END", st.START + st.TIME_DELTA)
st["SESSION_NAME"] = st.SESSION.apply(lambda x: SESSION_NAMES[x])
st.set_index("SESSION", inplace=True)

In [72]:
st.head()

,START,END,TIME_DELTA,SESSION_NAME
SESSION,,,,
Reg,2025-06-23 07:30:00,2025-06-23 08:20:00,0 days 00:50:00,Registration
OR,2025-06-23 08:20:00,2025-06-23 08:30:00,0 days 00:10:00,Opening Remarks
P1,2025-06-23 08:30:00,2025-06-23 09:30:00,0 days 01:00:00,Plenary Talk
P2,2025-06-23 09:30:00,2025-06-23 10:30:00,0 days 01:00:00,Plenary Talk
CB1,2025-06-23 10:30:00,2025-06-23 11:00:00,0 days 00:30:00,Coffee Break


In [80]:
st.START.dt.strftime("%d")

SESSION
Reg      23
OR       23
P1       23
P2       23
CB1      23
S1       23
LB1      23
S2       23
CB2      23
S3       23
Rec      23
P3       24
P4       24
CB3      24
S4       24
LB2      24
S5       24
CB4      24
S6       24
P5       25
P6       25
Photo    25
CB5      25
S7       25
LB3      25
BM       25
E        25
Ban      25
P7       26
P8       26
CB6      26
S8       26
LB4      26
S9       26
CB7      26
S10      26
S11      27
CB8      27
P9       27
P10      27
CR       27
Name: START, dtype: object

### Required functions

In [3]:
import pandas as pd

def cjken(s, prefix=""):
    """
    Input:
        s: string
    Output:
        enclose Chinese with \CJKEN{...}
    """
    if s != "":
        return prefix + r"\CJKEN{%s}"%s
    else:
        return ""

def write_a_session(s):
    want = sdf.loc[ sdf.index.str.startswith(s) ]
    l = want.shape[0]

    f.write(r"""\noindent\textbf{\CJKEN{場次} %s.} %s"""%(s[1],session_periods[s]) + '\n\n')
    f.write(r"""\begin{description}"""+'\n')
    for k in range(1,l+1):
        code = s+'%s'%k
        the_id = want.loc[code,'id']
        the_name = want.loc[code,'name']
        the_title = want.loc[code,'title']
        f.write(r"""\item[%s] \hyperlink{%s}{%s}\hypertarget{%s}{}, \textit{%s}"""%(
            talk_times[code], the_id, the_name, the_id+'top', the_title)+'\n')
    f.write(r"""\end{description}"""+'\n\n')
    
def write_mtskip():
    f.write(r"""\mtskip"""+'\n\n')
    
def write_btskip():
    f.write(r"""\btskip"""+'\n\n')
    
def write_newpage():
    f.write(r"""\newpage"""+'\n\n')
    
def write_teabreak():
    f.write(r"""\begin{description}
    \item[] \CJKEN{休息、茶點}
    \end{description}
    """)

def write_a_talk(file, data, ind, email=True, advisor=False):
    file.write(r'''{\parindent=0pt'''+'\n\n')
    file.write(r'''\hypertarget{'''+ind+r'''}{\talktitle{'''+data.loc[ind,'title']+'}}\n\n\\nopagebreak{}')
    file.write(r'''\centerline{%s, %s \hfill \hyperlink{%s}{$\Uparrow$}}'''%(
        data.loc[ind,'name'],data.loc[ind,'affiliation'],ind+'top')+'\n\n')
    if email:
        file.write(", ".join(r"""\url{%s}"""%addrs for addrs in data.loc[ind,'email'].split(",")) + '\n\n')
    if advisor:
        file.write(r"""Advisor: %s"""%data.loc[ind,'advisor'] + '\n\n')
    file.write('}\n\n\\nopagebreak{}')
    file.write(r'''\vspace{0.5cm}'''+'\n\n\\nopagebreak{}')
    file.write(data.loc[ind,'abstract']+'\n\n')
    file.write(r'''\btskip'''+'\n\n')
    
def write_name_n_title(file, data, advisor=False):
    file.write(r'''{\parindent=0pt'''+'\n\n')
    for ind in data.index:
        file.write(r'''\hypertarget{%s}\centerline{%s, %s \hfill \hyperlink{%s}{$\Uparrow$}}'''%(
            ind, 
            data.loc[ind,'name'], 
            data.loc[ind,'title'], 
            ind+'top')+'\n\n')
        file.write(r"""Advisor: %s"""%data.loc[ind,'advisor'] + '\n\n')
    file.write('}')

<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
/tmp/ipykernel_10494/4178683661.py:4: SyntaxWarning: invalid escape sequence '\C'
  """


### Schedule

In [102]:
##### CREATE SCHEDULE.TEX #####
f = open('schedule.tex','w')

### beginning
f.write(r"""\documentclass[ILAS2025-program.tex]{subfiles}

\begin{document}

""")

days = pd.date_range("2025-06-23", "2025-06-27")

for day in days:
    f.write(r"""\section{%s}
    
    \begin{description}
    """%day.strftime("%A Schedule, %B %d"))
    
    today = st.loc[st.START.dt.strftime("%d") == "23"].copy()
    today["START_STR"] = today.START.dt.strftime("%H:%M")
    today["END_STR"] = today.END.dt.strftime("%H:%M")
    for i,row in today.iterrows():
        f.write(r"""\item[%s--%s] %s
        """%(row.START_STR, row.END_STR, row.SESSION_NAME))
    
    f.write(r"""\end{description}
    """)
    
    write_newpage()



### Saturday
# f.write(r"""\noindent\talktitle{8/20\CJKEN{（六）}}"""+'\n\n')
# f.write(r"""\begin{description}
# \item[09:00--09:40] \CJKEN{報到、茶點}
# \item[09:40--10:00] \CJKEN{開幕}
# \end{description}
# """)
# write_a_session('11')
# f.write(r"""\begin{description}
# \item[] \CJKEN{午餐}
# \end{description}
# """)
# write_a_session('12')
# f.write(r"""\begin{description}
# \item \CJKEN{合照、茶點}
# \end{description}
# """)
# # write_teabreak()
# write_a_session('13')
# write_mtskip()
# f.write(r"""\begin{description}
# \item[17:30--18:30] \CJKEN{自由討論、學術交流}
# \item[18:27] \CJKEN{西子灣日落（參考地圖 sunset view 位置）}
# \item[18:30] \CJKEN{晚宴（理學院四樓）}
# \end{description}
# """)
# write_newpage()

# ### Sunday
# f.write(r"""\noindent\talktitle{8/21\CJKEN{（日）}}"""+'\n\n')
# write_a_session('21')
# write_teabreak()
# write_a_session('22')
# f.write(r"""\begin{description}
# \item[] \CJKEN{午餐}
# \end{description}
# """)
# write_a_session('23')
# write_teabreak()
# write_a_session('24')
# write_mtskip()
# f.write(r"""\begin{description}
# \item[16:30--17:00] \CJKEN{頒獎、交接、閉幕}
# \end{description}
# """)
# write_newpage()


### ending
f.write(r"""\end{document}
""")

f.close()

In [ ]:
df = pd.read_csv('speakers.csv').set_index('id')
### combine first name and last name
df.insert(0, 'name_en', df['first name']+' '+df['last name'])
df['name_en'].fillna("", inplace=True)
df['name_zh'].fillna("", inplace=True)
spaces = pd.Series([""] * df.shape[0], index=df.index)
spaces[ (df['name_en'] != "") & (df['name_zh'] != "") ] = " "
df.insert(0, "name", df['name_en'] + spaces + df['name_zh'].apply(cjken) )
### fill_na with 'Not yet received'
df['title'].fillna('To be announced', inplace=True)
df['abstract'].fillna('To be announced.', inplace=True)
### change data type of the index
df.index = df.index.astype('str')
### change data type of the column 'time code'
df['time code'] = df['time code'].astype('str')
### categorized into plenary and 25-min
df_plenary = df.loc[ df['type'] == 'plenary' ]
df_25min = df.loc[ (df['type'] == 'invited') | (df['type'] == 'contributed') ]
df_seedling = df.loc[ df['type'] == 'seedling' ]

sdf = df.reset_index().set_index('time code')

In [ ]:
session_periods = {}
talk_times = {}

session_periods['11'] = '10:00--12:00'
session_periods['12'] = '13:00--15:00'
session_periods['13'] = '15:30--17:30'
session_periods['21'] = '09:00--10:00'
session_periods['22'] = '10:30--12:00'
session_periods['23'] = '13:00--15:00'
session_periods['24'] = '15:30--16:30'

talk_times['111'] = '10:00--11:00'
talk_times['112'] = '11:00--11:30'
talk_times['113'] = '11:30--12:00'
talk_times['121'] = '13:00--14:00'  
talk_times['122'] = '14:00--14:30'    
talk_times['123'] = '14:30--15:00'    
talk_times['131'] = '15:30--16:00'    
talk_times['132'] = '16:00--16:30'    
talk_times['133'] = '16:30--17:00'
talk_times['134'] = '17:00--17:10'
talk_times['135'] = '17:10--17:20'
# talk_times['136'] = '17:20--17:30'

talk_times['211'] = '09:00--10:00'
talk_times['221'] = '10:30--11:00'
talk_times['222'] = '11:00--11:30'
talk_times['223'] = '11:30--12:00'
talk_times['231'] = '13:00--13:30'
talk_times['232'] = '13:30--14:00'
talk_times['233'] = '14:00--14:30'
talk_times['234'] = '14:30--15:00'
talk_times['241'] = '15:30--16:00'
talk_times['242'] = '16:00--16:30'
# talk_times['243'] = '16:30--17:00'
# talk_times['244'] = '17:00--17:30'
# talk_times['245'] = '17:30--18:00'

#session_periods
#talk_times

In [ ]:
##### CREATE SCHEDULE.TEX #####
f = open('schedule.tex','w')

### beginning
f.write(r"""\documentclass[NSYC2022-program.tex]{subfiles}

\begin{document}
""")


f.write(r"""\section{\CJKEN{議程}}
\begin{itemize}
\item[*] \CJKEN{地點：理學院國際會議廳}
\item[*] $\text{60 \CJKEN{分鐘演講}} = \text{50 \CJKEN{分鐘報告}} + \text{10 \CJKEN{討論}}$
\item[*] $\text{30 \CJKEN{分鐘演講}} = \text{20 \CJKEN{分鐘報告}} + \text{10 \CJKEN{討論}}$
\end{itemize}
""")

### Saturday
f.write(r"""\noindent\talktitle{8/20\CJKEN{（六）}}"""+'\n\n')
f.write(r"""\begin{description}
\item[09:00--09:40] \CJKEN{報到、茶點}
\item[09:40--10:00] \CJKEN{開幕}
\end{description}
""")
write_a_session('11')
f.write(r"""\begin{description}
\item[] \CJKEN{午餐}
\end{description}
""")
write_a_session('12')
f.write(r"""\begin{description}
\item \CJKEN{合照、茶點}
\end{description}
""")
# write_teabreak()
write_a_session('13')
write_mtskip()
f.write(r"""\begin{description}
\item[17:30--18:30] \CJKEN{自由討論、學術交流}
\item[18:27] \CJKEN{西子灣日落（參考地圖 sunset view 位置）}
\item[18:30] \CJKEN{晚宴（理學院四樓）}
\end{description}
""")
write_newpage()

### Sunday
f.write(r"""\noindent\talktitle{8/21\CJKEN{（日）}}"""+'\n\n')
write_a_session('21')
write_teabreak()
write_a_session('22')
f.write(r"""\begin{description}
\item[] \CJKEN{午餐}
\end{description}
""")
write_a_session('23')
write_teabreak()
write_a_session('24')
write_mtskip()
f.write(r"""\begin{description}
\item[16:30--17:00] \CJKEN{頒獎、交接、閉幕}
\end{description}
""")
write_newpage()


### ending
f.write(r"""\end{document}
""")

f.close()

In [ ]:
!pdflatex NSYC2022-program.tex

In [ ]:
##### CREATE CONTENT.TEX #####
f = open('content.tex','w')

### beginning
f.write(r"""\documentclass[NSYC2022-program.tex]{subfiles}

\begin{document}
""")

### plenary
f.write(r"""\section{\CJKEN{大會邀請演講}}
""")

f.write(r"""\begin{itemize}
\item[*] \CJKEN{依姓名拼音順序排列，各場報告時間請見議程}
\end{itemize}
""")


indices = df_plenary.index
for ind in indices:
    write_a_talk(f, df_plenary, ind, email=True, advisor=False)

f.write(r'''
\newpage

''')

### contributed
f.write(r"""\section{\CJKEN{學生演講}}
""")

f.write(r"""\begin{itemize}
\item[*] \CJKEN{依姓名拼音順序排列，各場報告時間請見議程}
\end{itemize}
""")

indices = df_25min.index
for ind in indices:
    write_a_talk(f, df_25min, ind, email=True, advisor=True)

f.write(r'''
\newpage

''')

### seedling
f.write(r"""\section{\CJKEN{種苗報告}}
""")

f.write(r"""\begin{itemize}
\item[*] \CJKEN{依姓名拼音順序排列，各場報告時間請見議程}
\end{itemize}
""")

write_name_n_title(f, df_seedling)

f.write(r'''
\newpage

''')

### ending
f.write(r"""\end{document}
""")

f.close()

In [ ]:
!pdflatex NSYC2022-program.tex

In [ ]:
df['name'].loc[df['abstract'] == 'To be announced.']

In [ ]:
df['name'].loc[df['abstract'] == 'only pdf received']

In [ ]:
(df['abstract'] == 'To be announced.').sum()

In [ ]:
(df['abstract'] == 'only pdf received').sum()

In [ ]:
(df['title'] == 'To be announced').sum()

In [ ]:
df

### Notes

It is not easy to handle missing Chinese fonts by the CJK package.  
Basically, one have to get the character image and the include it manually.  
For getting the picture:  

- Download it from [全字庫](https://www.cns11643.gov.tw/index.jsp) --- easy but the resolution seems low?
- If the character shows no problem on your local machine (you can check this by 全字庫), then you can use `fc-list | grep -i kai` or to search for your font; my case the font is stored at `/usr/share/fonts/TTF/TW-Kai-98_1.ttf` .  Then install `fontforge`, find the character by its UNICODE, double-click on it, then File > Export. 

Then you may include the picture by something like `includegraphics[scale=0.01]{fonts/filename}` .

```Python
s = '21'
want = sdf.loc[ sdf.index.str.startswith('21') ]
l = want.shape[0]

f.write(r"""\noindent\textbf{Session %s.} %s"""%(s[1],session_periods[s]) + '\n\n')
f.write(r"""\begin{description}"""+'\n')
for k in range(1,l+1):
    code = s+'%s'%k
    f.write(r"""\item[%s] %s, \textit{%s}"""%(talk_times[code], want.loc[code,'name'], want.loc[code,'title']))
f.write(r"""\end{description}"""+'\n\n')
```

Back up for plenary talks
``` Python
### plenary
f.write(r"""\section{Plenary Lectures}
""")

indices = df_plenary.index
for ind in indices:
    f.write(r'''\talktitle{'''+df_plenary.loc[ind,'title']+r'''}\label{'''+ind+'}\n\n')
    f.write(df_plenary.loc[ind,'name']+', '+df_plenary.loc[ind,'affiliation']+'\n\n')
    f.write(r'''\medskip'''+'\n\n')
    f.write(df_plenary.loc[ind,'abstract']+'\n\n')
    f.write(r'''\btskip'''+'\n\n')
```

```Python
def write_a_talk(file, data, ind):
    file.write(r'''{\parindent=0pt'''+'\n\n')
    file.write(r'''\talktitle{'''+data.loc[ind,'title']+r'''}\label{'''+ind+'}\n\n')
    file.write(data.loc[ind,'name']+', '+data.loc[ind,'affiliation']+'\n\n')
    file.write('}\n\n')
    file.write(r'''\vspace{0.5cm}'''+'\n\n')
    file.write(data.loc[ind,'abstract']+'\n\n')
    file.write(r'''\btskip'''+'\n\n')
 ```